# AWS Bedrock Tutorial

Amazon Bedrock is a fully managed service that provides access to foundation models from various AI providers through a single API.

In this notebook we'll:
1. Use the **Converse API** - the simplest way to interact with Bedrock models
2. Compare two models on the same prompt:
   - **Amazon Nova Micro** - Amazon's fastest/cheapest text model
   - **Mistral 7B** - Popular open-source model

## Setup

Create a Bedrock client. Credentials are picked up automatically from `~/.aws/credentials`.

In [ ]:
import boto3

bedrock = boto3.client("bedrock-runtime", region_name="us-east-1")

## Helper Function

A simple wrapper around the Converse API that also calculates cost.

In [ ]:
# Pricing per 1K tokens (check aws.amazon.com/bedrock/pricing for current rates)
PRICING = {
    "amazon.nova-micro-v1:0": {"input": 0.000035, "output": 0.00014},
    "mistral.mistral-7b-instruct-v0:2": {"input": 0.00015, "output": 0.0002},
}


def ask_bedrock(model_id, prompt, max_tokens=512, temperature=0.7):
    """Send a prompt to Bedrock and return the response."""
    response = bedrock.converse(
        modelId=model_id,
        messages=[{"role": "user", "content": [{"text": prompt}]}],
        inferenceConfig={"maxTokens": max_tokens, "temperature": temperature},
    )

    text = response["output"]["message"]["content"][0]["text"]
    usage = response["usage"]

    # Calculate cost
    price = PRICING[model_id]
    cost = (usage["inputTokens"] * price["input"] + usage["outputTokens"] * price["output"]) / 1000

    print(f"Tokens: {usage['inputTokens']} in, {usage['outputTokens']} out")
    print(f"Cost: ${cost:.6f}")

    return text

## Test with Amazon Nova Micro

In [ ]:
prompt = "Write a haiku about data engineering"

print("=== Amazon Nova Micro ===")
print(ask_bedrock("amazon.nova-micro-v1:0", prompt))

## Test with Mistral 7B

In [ ]:
print("=== Mistral 7B ===")
print(ask_bedrock("mistral.mistral-7b-instruct-v0:2", prompt))

## Compare the Results

Both models received the same prompt. Notice:
- **Token usage**: Different models tokenize text differently
- **Cost**: Nova Micro is significantly cheaper
- **Style**: Each model has its own "voice"

Try changing the prompt and running both cells again to see how the models compare on different tasks!